# Ch6 - Possible problem in AI training and Solution

In [15]:
import numpy as np
import matplotlib.pyplot as plt

(List the possible problems & solution encountered during training AI network)

## Neural network training

### Introduction

In the following of the notebook, we are using the *Iris flower dataset* for training.
(Explanation on the data, the components of data, no of data)

(The structure of the AI model, activation function & output layer function in between, optimization methods and loss function, batch size, test case,  )

### Data preparation

Below are the steps of data preparation,

1. Load the data from *sklearn* module
2. Normalization
3. Convert the label for training (Understand why do we need to do this)
4. Split the train and test case

In [16]:
from sklearn import datasets

iris_data = datasets.load_iris()
input_data = iris_data.data
correct = iris_data.target

no_of_data = len(correct)

In [17]:
# Print the first 10 data & take a look of the data
print(input_data[:10,])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]


In [18]:
# Normalization
mean_col = np.mean(input_data,axis=0)
std_col = np.std(input_data,axis=0)
normalized_data = (input_data - mean_col)/(std_col)

Turn the `correct` to one-hot labelling (Add the graph of one-hot labelling scheme & why do we need to turn it to one-hot)

In [19]:
# Turn the `correct` to one-hot labelling
correct_data = np.zeros((no_of_data,3),dtype=int)

# TODO: Figure out a way to make use the vectorization
for i in range(no_of_data):
    correct_data[i][correct[i]] = 1


In [20]:
# Split the data to test & learn case
from sklearn.model_selection import train_test_split

input_train, input_test, correct_train, correct_test = train_test_split(
    normalized_data,
    correct_data,
    test_size=no_of_data//2
)

n_train = no_of_data//2

### Network structure

1. Define the hyperparameters & initial vaules for the network
2. Define the layer structure as object
3. Forward and back propagation definition


In [21]:
# Nework neuron
n_in = 4 # since we get 4 features for each x
n_mid = 25
n_out = 3

eta = 0.01      # learning rate for SGD
epoch = 1000    # No of training times
batch_size = 8

interval = 100  # Showing the training accuracy every (interval) times

(ReLu definition, Softmax expression)

In [34]:
# Define the layer structure & methods


# Define the common properities in BaseLayer
class BaseLayer:
  def __init__(self,n_upper,n, wb_width = 0.1): # n_upper - no of neurons in last layer, n - no of neurons
    self.w = wb_width * np.random.randn(n_upper, n)
    self.b = wb_width * np.random.randn(n)

  def update(self,eta):
    self.w -= eta * self.grad_w
    self.b -= eta * self.grad_b

# Middle layer
class MiddleLayer(BaseLayer):
  def forward(self, x):
    self.x = x
    self.u = np.dot(x, self.w) + self.b
    self.y = np.where(self.u<=0, 0, self.u) # ReLu activation function ]

  def backward(self,grad_y):
    delta = grad_y * np.where(self.u>0, 1, 0) # grad_y * derivative of ReLu

    self.grad_w = np.dot(self.x.T, delta)
    self.grad_b = np.sum(delta, axis=0)

    self.grad_x = np.dot(delta, self.w.T)

# Output layer
class OutputLayer(BaseLayer):
  def forward(self,x):
    self.x = x
    self.u = np.dot(x,self.w) + self.b
    self.y = np.exp(self.u)/(np.sum(self.u,axis=1,keepdims=True))

  def backward(self,t):
    delta = self.y - t

    self.grad_w = np.dot(self.x.T, delta)
    self.grad_b = np.sum(delta, axis=0)

    self.grad_x = np.dot(delta, self.w.T)


In [23]:
# Neural network structure

middle_layer_1 = MiddleLayer(n_in, n_mid)
middle_layer_2 = MiddleLayer(n_mid, n_mid)
output_layer = OutputLayer(n_mid,n_out)

In [24]:
# Group the forward, backward propagation, update_weight of different layer in the function

def forward_propagation(x):
  middle_layer_1.forward(x)
  middle_layer_2.forward(middle_layer_1.y)
  output_layer.forward(middle_layer_2.y)

def backward_propagation(t):
  output_layer.backward(t)
  middle_layer_2.backward(output_layer.grad_x)
  middle_layer_1.backward(middle_layer_2.grad_x)

def update_wb(eta):
  middle_layer_1.update(eta)
  middle_layer_2.update(eta)
  output_layer.update(eta)

def get_error(t, batch_size):
  # Basically calcaluating the cost function value and normalize with the batch size
  # add 1e-7 to avoid ln(0)
  return -np.sum(-t * np.log(output_layer.y + 1e-7))/batch_size



### Training procedures

In [37]:
train_error = []

test_error = []

no_of_batch = n_train // batch_size

for i in range(epoch):

  # Record the error on training & test case in every training
  forward_propagation(input_train)
  train_error.append(get_error(correct_train, batch_size))

  forward_propagation(input_test)
  test_error.append(get_error(correct_test, batch_size))

  # Shuffle the data & prepare the index
  index_random = np.arange(n_train)
  np.random.shuffle(index_random)

  for j in range(no_of_batch):

    mini_batch_index = index_random[j*batch_size:(j+1)*batch_size]
    x = input_train[mini_batch_index, :]
    t = correct_train[mini_batch_index, :]

    forward_propagation(x)
    backward_propagation(t)
    update_wb(eta)

<ipython-input-24-1f8081bbb656>:21: RuntimeWarning: invalid value encountered in log
  return -np.sum(-t * np.log(output_layer.y + 1e-7))/batch_size


AttributeError: 'MiddleLayer' object has no attribute 'b_'